In [1]:
from flask import Flask, jsonify, render_template, request
from flask_mysqldb import MySQL
import random
import requests
import tensorflow
import csv
import os
import speech_recognition as sr
import numpy as np
import librosa 
import pandas as pd
from keras.models import load_model
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from keras import backend as K
from datetime import datetime
import mysql.connector as connector
import pymysql
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import csv
import scipy
from scipy import signal

from IPython.display import Audio
from scipy.io import wavfile

In [2]:

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

dependencies = {'f1_m': f1_m }
# create a custom function to load model
def load_all_models(n_models):
    all_models = list()
    for i in range(n_models):
		# Specify the filename 
        filename = 'Ensemble_Models\model'+ str(i + 1) + '.h5'
        # load the model 
        model = load_model(filename,custom_objects=dependencies)
		# Add a list of all the weaker learners
        all_models.append(model)
        print('>loaded %s' % filename)
    return all_models

# create stacked model input dataset as outputs from the ensemble
def stacked_dataset(members, inputX):
    stackX = None
    for model in members:
        # make prediction
        yhat = model.predict(inputX, verbose=0)
        # stack predictions into [rows, members, probabilities]
        if stackX is None:
            stackX = yhat #
        else:
            stackX = np.dstack((stackX, yhat))
    # flatten predictions to [rows, members x probabilities]
    stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
    return stackX

def fit_stacked_model(members, inputX, inputy):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX)
    # fit the meta learner
    model = LogisticRegression() #meta learner
    model.fit(stackedX, inputy)
    return model

def stacked_prediction(members, model, inputX):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX)
    # make a prediction
    yhat = model.predict(stackedX)
    return yhat

def noise(data):
    noise_amp = 0.035*np.random.uniform()*np.amax(data)
    data = data + noise_amp*np.random.normal(size=data.shape[0])
    return data

def feat_ext(data):
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=22050).T, axis=0)
    return mfcc

def get_feat(path):
    data, sample_rate = librosa.load(path, duration=5, offset=0.6)
    # normal data
    res1 = feat_ext(data)
    result = np.array(res1)
    #data with noise
    noise_data = noise(data)
    res2 = feat_ext(noise_data)
    result = np.vstack((result, res2))
    return result

# Find the name of the class with the top score when mean-aggregated across frames.
def class_names_from_csv(class_map_csv_text):
  """Returns list of class names corresponding to score vector."""
  class_names = []
  with tf.io.gfile.GFile(class_map_csv_text) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
      class_names.append(row['display_name'])

  return class_names

def ensure_sample_rate(original_sample_rate, waveform,
                       desired_sample_rate=16000):
  """Resample waveform if required."""
  if original_sample_rate != desired_sample_rate:
    desired_length = int(round(float(len(waveform)) /
                               original_sample_rate * desired_sample_rate))
    waveform = scipy.signal.resample(waveform, desired_length)
  return desired_sample_rate, waveform

In [3]:
app = Flask(__name__, template_folder='template', static_folder='static')

n_members = 4
members = load_all_models(n_members)
print('Loaded %d models' % len(members))

# app.config['MYSQL_HOST'] = 'localhost'
# app.config['MYSQL_PORT'] = '3306'
# app.config['MYSQL_USER'] = 'root'
# app.config['MYSQL_PASSWORD'] = 'Ankita@040899'
# app.config['MYSQL_DB'] = 'sih'
# mysql = MySQL(app)

connection = pymysql.connect(host='localhost', user='root', password='Ankita@040899', database='sih', port = 3306)

>loaded Ensemble_Models\model1.h5
>loaded Ensemble_Models\model2.h5
>loaded Ensemble_Models\model3.h5
>loaded Ensemble_Models\model4.h5
Loaded 4 models


In [4]:
# Load the model.
bgmodel = hub.load('https://tfhub.dev/google/yamnet/1')

In [5]:
@app.route('/')
def home():
    return render_template('index.html')

In [6]:
@app.route('/audio/')
def audio_to_text():
    return render_template('audio.html')

In [7]:
@app.route('/history')
def history():
    return render_template('ser_database.html')

In [8]:
@app.route('/predict',methods=['GET','POST'])
def predict():

    
    with open('upload/audio.wav', 'wb') as f:
        f.write(request.data)
    try:
        r = sr.Recognizer()
        # open the file
        with sr.AudioFile('upload/audio.wav') as source:
        # listen for the data (load audio to memory)
            # r.adjust_for_ambient_noise(source)
            audio_data = r.listen(source)
        # recognize (convert from speech to text)
            text = r.recognize_google(audio_data)
            print(text.casefold())
            speech_text = text.casefold()
    except sr.UnknownValueError:
        return str("Blank Call")

    class_map_path = bgmodel.class_map_path().numpy()
    class_names = class_names_from_csv(class_map_path) 

        # wav_file_name = 'speech_whistling2.wav'
    wav_file_name = 'upload/audio.wav'
    sample_rate, wav_data = wavfile.read(wav_file_name, 'rb')
    sample_rate, wav_data = ensure_sample_rate(sample_rate, wav_data)   
    waveform = wav_data / tf.int16.max
    # Run the model, check the output.
    scores, embeddings, spectrogram = bgmodel(waveform)

    scores_np = scores.numpy()
    spectrogram_np = spectrogram.numpy()
    infered_class = class_names[scores_np.mean(axis=0).argmax()]
    print(f'The main sound is: {infered_class}')
        
    Features = pd.read_csv(r'features_Emergency.csv')
    X = Features.iloc[:,:-1].values
    Y = Features['labels'].values
    encoder = OneHotEncoder()
    Y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()

    # with open('Words_emotion.csv', 'rb') as file:
    #     dataset = file.read()


    # splitting data
    x_train, x_test, y_train, y_test = train_test_split(X, Y,test_size=0.3, random_state=0, shuffle=True)
    
    x_traincnn =np.expand_dims(x_train, axis=2)
    x_testcnn= np.expand_dims(x_test, axis=2)
    
    y_test = encoder.inverse_transform(y_test)

    model = fit_stacked_model(members,x_testcnn, y_test)	
    
    feature = get_feat('upload/audio.wav')
    test =np.expand_dims(feature, axis=2)
    livepreds = stacked_prediction(members, model, test)
    print (livepreds)

    dataset = pd.read_csv(r'WORDS_EMOTIONS.csv')
    # dataset = dataset.decode('ISO-8859-1', 'ignore')
    dataset.astype({'Phrases':'string', 'Emotions':'string'}).dtypes
    X = dataset['Phrases']
    y = dataset['Emotions']
    emotion_list=[]
    for i in range(len(X)):
        if X[i] in speech_text:
            emotion_list.append(y[i])  
    
    # if len(emotion_list)>0:
    try:
        count_angry=emotion_list.count("abusive")
        count_notprank=emotion_list.count("Not_Prank")
        count_prank=emotion_list.count("prank")
        total = count_prank+count_angry+count_notprank
        angry_percentage = count_angry/total*100
        prank_percentage = count_prank/total*100
        notprank_percentage = count_notprank/total*100
        count_list = [angry_percentage,notprank_percentage ,prank_percentage]
        for ind, i in enumerate(count_list):
            count_list[ind] = "{}%".format(i)
        result_list = ['Abusive','Not Prank','Prank']
        df = pd.DataFrame(zip(result_list,count_list))
    except ZeroDivisionError:
        count_list = ['0%','0%','0%']
        # for ind, i in enumerate(count_list):
            # [ind] = "{}%".format(i)
        result_list = ['Abusive','Not Prank','Prank']
        df = pd.DataFrame(zip(result_list,count_list))

    final_df = df.sort_values(by=[0], ascending=False)
    final = final_df.iloc[0,:]+final_df.iloc[1,:]+ final_df.iloc[2,:]
    dic = dict()
    dic = {"Emotion":livepreds[0],final_df.iloc[0,0]:final_df.iloc[0,1], final_df.iloc[1,0]:final_df.iloc[1,1], final_df.iloc[2,0]:final_df.iloc[2,1],"Main Sound":infered_class}
    dt=datetime.now()
    # ser_result = str(livepreds[0])
    # not_prank = str(final_df.iloc[1,1])
    # prank = str(final_df.iloc[2,1])
    # abusive = str(final_df.iloc[0,1])
    # main_audio = str(livepreds[0])
    # cur = mysql.connection.cursor()
    cursor=connection.cursor()
    # print(prank)
    sql='INSERT INTO ser(ser_result, not_prank, prank, abusive, main_audio) VALUES (%s, %s, %s, %s, %s);'
    # val=(ser_result,not_prank,prank,abusive, main_audio)
    val=(str(livepreds[0]),str(final_df.iloc[1,1]),str(final_df.iloc[2,1]),str(final_df.iloc[0,1]), str(infered_class))
    cursor.execute(sql,val)
    # mysql.connection.commit()
    connection.commit()
    # cur.close()
    return str(dic) 


In [9]:
# @app.route('/results',methods=['POST'])
# def results():
#     data = request.get_json(force=True)
#     prediction = model.predict(data.values())
    
#     return jsonify(prediction)

In [10]:
if __name__ == "__main__":
    app.run(debug=True,use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [26/Aug/2022 17:52:40] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2022 17:52:40] "GET /static/css/indexStyle.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2022 17:52:40] "GET /static/css/images/logo.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2022 17:52:40] "GET /static/css/style.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2022 17:52:40] "GET /static/css/images/bgimg2.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2022 17:52:40] "GET /static/css/images/favicon.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2022 17:52:48] "GET /audio/ HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2022 17:52:48] "GET /static/js/try.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2022 17:52:48] "GE

can you call an ambulance as soon as possible
The main sound is: Speech


c:\Users\saxen\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
INFO:werkzeug:127.0.0.1 - - [26/Aug/2022 17:53:53] "POST /predict HTTP/1.1" 200 -


['painful' 'painful']


INFO:werkzeug:127.0.0.1 - - [26/Aug/2022 17:53:56] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2022 17:53:56] "GET /static/css/style.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2022 17:53:57] "GET /static/css/indexStyle.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2022 17:53:57] "GET /static/css/images/logo.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2022 17:53:57] "GET /static/css/images/bgimg2.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2022 17:53:58] "GET /audio/ HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2022 17:53:58] "GET /static/js/main.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2022 17:53:58] "GET /static/css/style.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2022 17:53:58] "GET /static/js/recorder.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2022 17:53:58] "GET /static/js/try.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2022 17:53:58] "GET /static/js/recordhelper.j